In [1]:
import itertools
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp,shapiro,levene,ttest_ind,mannwhitneyu,f_oneway,kruskal
from statsmodels.stats.proportion import proportions_ztest

## Görev 1 (Veriyi Hazırlama ve Analiz Etme)

#### Adım 1
- ab_testing_data.xlsxadlı kontrol ve test grubu verilerinden oluşan veri setini okutunuz. Kontrol ve test grubu verilerini ayrı değişkenlere atayınız.

In [2]:
dfcontrol = pd.read_excel(r'../input/abtesti/ab_testing.xlsx',sheet_name='Control Group')

dftest = pd.read_excel(r'../input/abtesti/ab_testing.xlsx', sheet_name='Test Group')

In [3]:
dftest.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [4]:
dfcontrol.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


#### Adım 2
- Kontrol ve test grubu verilerini analiz ediniz.

In [5]:
dfcontrol.head()
dfcontrol.shape
dfcontrol.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,101711.449068,20302.157862,45475.942965,85726.690349,99790.701078,115212.816543,147539.336329
Click,40.0,5100.657373,1329.985498,2189.753157,4124.304129,5001.220602,5923.803596,7959.125069
Purchase,40.0,550.894059,134.108201,267.028943,470.095533,531.206307,637.957088,801.795020
Earning,40.0,1908.568300,302.917783,1253.989525,1685.847205,1975.160522,2119.802784,2497.295218


In [6]:
dftest.head()
dftest.shape
dftest.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.0,120512.411758,18807.448712,79033.834921,112691.970770,119291.300775,132050.578933,158605.920483
Click,40.0,3967.549761,923.095073,1836.629861,3376.819024,3931.359804,4660.497911,6019.695079
Purchase,40.0,582.106097,161.152513,311.629515,444.626828,551.355732,699.862360,889.910460
Earning,40.0,2514.890733,282.730852,1939.611243,2280.537426,2544.666107,2761.545405,3171.489708


In [7]:
dfcontrol["GROUP"] = "control"
dftest["GROUP"] = "test"

In [8]:
dfcontrol.head()

,Impression,Click,Purchase,Earning,GROUP
0,82529.459271,6090.077317,665.211255,2311.277143,control
1,98050.451926,3382.861786,315.084895,1742.806855,control
2,82696.023549,4167.965750,458.083738,1797.827447,control
3,109914.400398,4910.882240,487.090773,1696.229178,control
4,108457.762630,5987.655811,441.034050,1543.720179,control


#### Adım 3
- Analiz işleminden sonra concatmetodunu kullanarak kontrol ve test grubu verilerini birleştiriniz.

In [9]:
df=pd.concat([dfcontrol,dftest])

In [10]:
df

,Impression,Click,Purchase,Earning,GROUP
0,82529.459271,6090.077317,665.211255,2311.277143,control
1,98050.451926,3382.861786,315.084895,1742.806855,control
2,82696.023549,4167.965750,458.083738,1797.827447,control
3,109914.400398,4910.882240,487.090773,1696.229178,control
4,108457.762630,5987.655811,441.034050,1543.720179,control
...,...,...,...,...,...
35,79234.911929,6002.213585,382.047116,2277.863984,test
36,130702.239410,3626.320072,449.824592,2530.841327,test
37,116481.873365,4702.782468,472.453725,2597.917632,test
38,79033.834921,4495.428177,425.359102,2595.857880,test


## GÖREV 2 : A/B Testinin Hipotezinin Tanımlanması

#### Adım 1
- Hipotezi tanımlayınız.

In [11]:
#H0 :  Kontrol ve Test grubu satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark yoktur.
#H0 : M1 = M2
#H1 : M1!= M2

#### Adım 2 
- Kontrol ve test grubu için Purchase(kazanç) ortalamalarını analiz ediniz.

In [12]:
df.groupby("GROUP").agg({"Purchase":[np.mean]})

,Purchase
,mean
GROUP,
control,550.894059
test,582.106097


## GÖREV 3: Hipotez Testinin Gerçekleştirilmesi

#### Adım 1
- Hipotez testi yapılmadan önce varsayım kontrollerini yapınız.

In [13]:
# Normallik Varsayımı(control için):
test_stat,pvalue=shapiro(df.loc[df["GROUP"]=='control',"Purchase"])
print("Test Stat=%.4f,p-value = %.4f"%(test_stat,pvalue))

Test Stat=0.9773,p-value = 0.5891


In [14]:
# p-value > 0.05 
# H0: reddeddilmez.Normallik sağlanır.

In [15]:
# Normallik Varsayımı(test için)
test_stat,pvalue=shapiro(df.loc[df["GROUP"]=="test","Purchase"])
print("Test Stat=%.4f  , p-value = %.4f"%(test_stat,pvalue))

Test Stat=0.9589  , p-value = 0.1541


In [16]:
# p-value >0.05
# H0: Reddedilmez.Normallik sağlanır.

In [17]:
# Varyans Homojenliği
test_stat,pvalue=levene(df.loc[df["GROUP"]=="control","Purchase"],
                        df.loc[df["GROUP"]=="test","Purchase"])
print("Test Stat=%.4f , p-value=%.4f"%(test_stat,pvalue))


Test Stat=2.6393 , p-value=0.1083


In [18]:
# p-value > 0.05
# H0:Reddedilmez.Varyans homojenliği sağlanır.

In [19]:
test_stat,pvalue=ttest_ind(df.loc[df["GROUP"]=="control","Purchase"],
                        df.loc[df["GROUP"]=="test","Purchase"])
print("Test Stat=%.4f , p-value=%.4f"%(test_stat,pvalue))

Test Stat=-0.9416 , p-value=0.3493


In [20]:
# p-value = 0.3493 olduğu için H0 REDDEDİLEMEZ!
# H0 :  kontrol ve test grubu satın alma ortalamaları arasında istatistiki olarak anlamlı bir fark YOKTUR.
# Yani fark şans eseri ortaya çıkmıştır.
